In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [6]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.66, 1.66, 1.85)]
  wgt = [(0.5, 0.55, 0.85)]


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312      1.0 -1.062290e-05                  NaN           NaN
0.000312      NaN           NaN                  1.0 -1.944353e-05
0.000750      2.0 -1.059140e-05                  2.0 -3.984327e-05
0.001052      3.0 -1.050866e-05                  3.0 -1.141398e-05
0.001476      4.0 -1.041045e-05                  4.0 -8.809261e-06
0.002070      5.0 -1.030669e-05                  5.0 -8.622156e-06
0.002904      6.0 -1.018848e-05                  6.0 -8.633349e-06
0.004074      7.0 -1.006336e-05                  7.0 -8.458914e-06
0.005714      8.0 -9.925574e-06                  8.0 -8.515187e-06
0.008015      9.0 -9.775611e-06                  9.0 -8.365896e-06
0.011243     10.0 -9.615750e-06                 10.0 -8.412427e-06
0.015771     11.0 -9.442848e-06                 11.0 -8.349335e-06
0.022122     12.0 -9.257462e-06                 12.0 -8.363300e-06
0.031031     13.0 -9.059223e-06                 13.0 -8.431593e-06
0.043528     14.0 -8.832380e-06                 14.0 -1.090471e-05
0.061057     15.0 -8.069688e-06                 15.0 -1.337432e-05
0.085645     16.0 -5.208693e-06                 16.0 -1.082055e-05
0.120136     17.0  1.535458e-06                 17.0 -6.497499e-06
0.168516     18.0  1.316966e-05                 18.0 -2.120440e-06
0.236378     19.0  3.193888e-05                 19.0  4.992451e-06
0.331549     20.0  6.115555e-05                 20.0  1.625682e-05
0.465100     21.0  1.054301e-04                 21.0  3.369284e-05
0.652400     22.0  1.706203e-04                 22.0  6.238793e-05
0.915100     23.0  2.419800e-04                 23.0  9.448516e-05
1.283650     24.0  2.483458e-04                 24.0  9.623369e-05
1.800600     25.0  1.899922e-04                 25.0  7.082166e-05
2.525700     26.0  1.311518e-04                 26.0  4.629873e-05
3.542800     27.0  8.592346e-05                 27.0  2.774946e-05
4.969550     28.0  5.357845e-05                 28.0  1.471627e-05
6.970850     29.0  3.166481e-05                 29.0  6.019202e-06
9.778100     30.0  1.684127e-05                 30.0  2.057384e-07
13.715850    31.0  6.401899e-06                 31.0 -3.861472e-06
19.239350    32.0 -8.125351e-07                 32.0 -6.640706e-06
26.987250    33.0 -5.081734e-06                 33.0 -8.308425e-06
37.855300    34.0 -7.531344e-06                 34.0 -9.360107e-06
53.100050    35.0 -9.347076e-06                 35.0 -1.026629e-05
73.887500    36.0 -1.072693e-05                 36.0 -1.111021e-05
97.662500    37.0 -1.150162e-05                 37.0 -1.176568e-05
121.437500   38.0 -1.191089e-05                 38.0 -1.219728e-05
145.212500   39.0 -1.228885e-05                 39.0 -1.242396e-05
168.987500   40.0 -1.297994e-05                 40.0 -1.323153e-05
192.762500   41.0 -1.891988e-05                 41.0 -2.014434e-05
216.537500   42.0 -4.201969e-05                 42.0 -5.171385e-05
240.312500   43.0 -6.550691e-05                 43.0 -1.045167e-04
264.087500   44.0 -5.472968e-05                 44.0 -1.671103e-04
287.862500   45.0  6.201903e-05                 45.0 -1.991390e-04
311.637500   46.0  3.413592e-04                 46.0 -1.000308e-04
335.412500   47.0  8.052359e-04                 47.0  1.489605e-04
359.187500   48.0  1.520792e-03                 48.0  5.941571e-04
382.962500   49.0  2.554054e-03                 49.0  1.313494e-03
406.737500   50.0  3.967896e-03                 50.0  2.381069e-03
430.512500   51.0  5.816758e-03                 51.0  3.857769e-03
454.287500   52.0  7.985430e-03                 52.0  5.723670e-03
478.062500   53.0  1.052540e-02                 53.0  8.039838e-03
501.837500   54.0  1.353775e-02                 54.0  1.090596e-02
525.612500   55.0  1.706594e-02                 55.0  1.424836e-02
549.387500   56.0  2.115224e-02                 56.0  1.82197

# Fluxes by Layer

CRD                              CLIRAD  \
                         flug          fldg      fnetg       flug   
pressure     level                                                  
0.000000e+00 1     -60.494102  0.000000e+00 -60.494102        NaN   
1.000000e-08 1            NaN           NaN        NaN -60.533611   
6.244000e-04 2     -60.494102  6.370326e-10 -60.494102 -60.533611   
8.759000e-04 3     -60.494102  8.972731e-10 -60.494102 -60.533611   
1.228600e-03 4     -60.494102  1.273426e-09 -60.494102 -60.533611   
1.723400e-03 5     -60.494102  1.818278e-09 -60.494102 -60.533611   
2.417400e-03 6     -60.494102  2.607369e-09 -60.494102 -60.533611   
3.390900e-03 7     -60.494102  3.750226e-09 -60.494102 -60.533611   
4.756500e-03 8     -60.494102  5.405353e-09 -60.494102 -60.533611   
6.672000e-03 9     -60.494102  7.802086e-09 -60.494102 -60.533611   
9.358900e-03 10    -60.494102  1.127264e-08 -60.494102 -60.533611   
1.312780e-02 11    -60.494102  1.629780e-08 -60.494102 -60.533611   
1.841450e-02 12    -60.494102  2.357363e-08 -60.494102 -60.533611   
2.583020e-02 13    -60.494102  3.410766e-08 -60.494102 -60.533611   
3.623230e-02 14    -60.494102  4.935855e-08 -60.494102 -60.533611   
5.082340e-02 15    -60.494102  7.148286e-08 -60.494102 -60.533611   
7.129060e-02 16    -60.494102  1.057322e-07 -60.494102 -60.533611   
1.000000e-01 17    -60.494102  1.681206e-07 -60.494102 -60.533611   
1.402710e-01 18    -60.494102  2.941033e-07 -60.494102 -60.533611   
1.967600e-01 19    -60.494102  5.492817e-07 -60.494102 -60.533612   
2.759970e-01 20    -60.494102  1.063903e-06 -60.494101 -60.533612   
3.871000e-01 21    -60.494103  2.096643e-06 -60.494101 -60.533612   
5.431000e-01 22    -60.494103  4.161758e-06 -60.494099 -60.533612   
7.617000e-01 23    -60.494102  8.253091e-06 -60.494094 -60.533613   
1.068500e+00 24    -60.494101  1.575926e-05 -60.494085 -60.533613   
1.498800e+00 25    -60.494099  2.650447e-05 -60.494073 -60.533613   
2.102400e+00 26    -60.494098  3.875546e-05 -60.494059 -60.533613   
2.949000e+00 27    -60.494098  5.182260e-05 -60.494046 -60.533615   
4.136600e+00 28    -60.494099  6.552132e-05 -60.494034 -60.533617   
5.802500e+00 29    -60.494103  7.984694e-05 -60.494023 -60.533621   
8.139200e+00 30    -60.494110  9.501035e-05 -60.494015 -60.533626   
1.141700e+01 31    -60.494119  1.112812e-04 -60.494008 -60.533634   
1.601470e+01 32    -60.494133  1.287618e-04 -60.494005 -60.533645   
2.246400e+01 33    -60.494153  1.475854e-04 -60.494005 -60.533660   
3.151050e+01 34    -60.494179  1.687787e-04 -60.494011 -60.533680   
4.420010e+01 35    -60.494216  1.940433e-04 -60.494022 -60.533708   
6.200000e+01 36    -60.494266  2.246327e-04 -60.494042 -60.533746   
8.577500e+01 37    -60.494332  2.604892e-04 -60.494072 -60.533799   
1.095500e+02 38    -60.494399  2.942958e-04 -60.494104 -60.533853   
1.333250e+02 39    -60.494466  3.280904e-04 -60.494138 -60.533908   
1.571000e+02 40    -60.494534  3.620113e-04 -60.494172 -60.533965   
1.808750e+02 41    -60.494608  3.987956e-04 -60.494209 -60.534026   
2.046500e+02 42    -60.494727  4.650772e-04 -60.494262 -60.534126   
2.284250e+02 43    -60.495072  6.915167e-04 -60.494381 -60.534421   
2.522000e+02 44    -60.495909  1.344191e-03 -60.494565 -60.535157   
2.759750e+02 45    -60.497675  2.955661e-03 -60.494719 -60.536736   
2.997500e+02 46    -60.500998  6.453689e-03 -60.494545 -60.539750   
3.235250e+02 47    -60.505837  1.225408e-02 -60.493583 -60.544194   
3.473000e+02 48    -60.512261  2.094584e-02 -60.491315 -60.550110   
3.710750e+02 49    -60.520563  3.353065e-02 -60.487032 -60.557804   
3.948500e+02 50    -60.531044  5.120445e-02 -60.479839 -60.567614   
4.186250e+02 51    -60.544001  7.533618e-02 -60.468664 -60.579817   
4.424000e+02 52    -60.559727  1.074445e-01 -60.452283 -60.594642   
4.661750e+02 53    -60.578122  1.483289e-01 -60.429793 -60.612055   
4.899500e+02 54    -60.599206  1.990556e-01 -60.400150 -60.632135   
5.137250e+02 55    -60.6

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')